# 03 - Despliegue del Servicio Web en Azure
---
Este notebook contiene el despliegue de un modelo de regresión como API usando **Flask**. 

### Importamos Librerías

In [2]:
from flask import Flask, request, jsonify
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import joblib
import numpy as np
import json

### Creación de Workspace y Registro de Modelo

In [ ]:
# Cargando el modelo
id = open('id_confidential.json', 'r')
mi = json.load(id)
my_id = mi["azure_id"]

ws = Workspace.create(name="sales",
                      subscription_id = my_id,
                      resource_group = "sls",
                      location = "centralindia")

registered_model = Model.register(model_path="model.pkl",
                                  model_name="modifiedate_model",
                                  workspace=ws)

Deploying KeyVault with name saleskeyvaultee9ec7afe73.
Deploying AppInsights with name salesinsights1aab289be19.
Deployed AppInsights with name salesinsights1aab289be19. Took 10.14 seconds.
Deploying StorageAccount with name salesstorage3b669416ab4d.
Deployed KeyVault with name saleskeyvaultee9ec7afe73. Took 26.41 seconds.
Deployed StorageAccount with name salesstorage3b669416ab4d. Took 29.39 seconds.
Deploying Workspace with name sales.
Deployed Workspace with name sales. Took 45.79 seconds.


### Creación del ScorePY

In [ ]:
scorepy = """import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('modifiedate_model')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)

        # Seleccionar columnas relevantes
        data = data[['CompanyName', 'SalesPerson', 'ModifiedDate']]

        # Convertir ModifiedDate a datetime y luego a rango de días
        data['ModifiedDate'] = pd.to_datetime(data['ModifiedDate'])
        data['Fecha'] = (data['ModifiedDate'] - data['ModifiedDate'].min()).dt.days

        # Eliminar columna original
        data.drop(columns=['ModifiedDate'], inplace=True)

        # Realizar predicción usando solo la columna transformada
        result = model.predict(data[['Fecha']])

        return json.dumps({"result": result.tolist()})
    except Exception as e:
        return json.dumps({"error": str(e)})

"""

with open("score.py", "w") as file_score:
    file_score.write(scorepy)

### Configuración de Inferencia

In [ ]:
virtual_env = Environment("env_sales")

virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn'])

inference_config = InferenceConfig(environment=virtual_env,
                                   entry_script="score.py")

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

In [ ]:
service.wait_for_deployment()

### Creación del URI

In [ ]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [ ]:
print(scoring_uri)